In [ ]:
import numpy as np 
import pandas as pd 
import holidays

import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append(r'C:\Users\Emincan\Desktop\Gdz\gdz-competition\Önemli Scriptler')

In [ ]:
data = pd.read_csv('train.csv' , parse_dates=['Tarih'] )
submit = pd.read_csv('sample_submission.csv', parse_dates=['Tarih'])
med = pd.read_csv('med.csv' , parse_dates=['Tarih'])


calendar = pd.read_csv('Turkish_calendar.csv',sep=(";"))

# Feature Engineering

In [ ]:
from feat_engs import create_time_features1 , create_time_features2

# data , submit = create_time_features1(data , med , calendar ,submit_df)
data = create_time_features2(data , med)
submit = create_time_features2(submit , med)

data

In [ ]:
plt.figure(figsize = (16,20) , dpi=100)
sns.heatmap(data.corr(numeric_only=True).round(2) , annot = True)

In [ ]:
plt.figure(figsize = (16,12) , dpi=100)
sns.heatmap(submit.corr(numeric_only=True).round(2) , annot = True)

In [ ]:
# data.drop(['electrical_outage' , 'outage_percentage' , 'rolling_outages_24h' , 'exp_avg_outages_24h' , 'is_weekend' , 'is_winter' , 'is_weekday'] , axis = 1 , inplace=True)

In [ ]:
data.info()

# Train | Test Split

In [ ]:
data.set_index('Tarih' , inplace=True)
submit.set_index('Tarih' , inplace=True)

data['week_of_year'] = data['week_of_year'].astype('int')

In [ ]:
X=data.drop("Dağıtılan Enerji (MWh)", axis=1)
y=data["Dağıtılan Enerji (MWh)"]
# forecast=submit.drop("Dağıtılan Enerji (MWh)", axis=1)
forecast = submit.copy().drop('Dağıtılan Enerji (MWh)' , axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2 , random_state=53 , shuffle=False)

# Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_forecast_scaled = scaler.transform(forecast)

In [ ]:
X.shape

# Reshape and Packages

In [ ]:
def packager(X_scaled, X_train_scaled, X_test_scaled, X_forecast_scaled, window_size=24):
    X_3d, y_2d, X_train_3d, y_train_2d, X_test_3d, y_test_2d, X_forecast_3d = [], [], [], [], [], [], []
    
    for i in range(window_size, len(X_scaled)):
        X_3d.append(X_scaled[i-window_size:i, :])
        y_2d.append(X_scaled[i, 0])

    for i in range(window_size, len(X_train_scaled)):
        X_train_3d.append(X_train_scaled[i-window_size:i, :])
        y_train_2d.append(X_train_scaled[i, 0])

    for i in range(window_size, len(X_test_scaled)):
        X_test_3d.append(X_test_scaled[i-window_size:i, :])
        y_test_2d.append(X_test_scaled[i, 0])
        
    for i in range(window_size, len(X_forecast_scaled)):
        X_forecast_3d.append(X_forecast_scaled[i-window_size:i, :])

    X_3d, y_2d = np.array(X_3d), np.array(y_2d)
    X_train_3d, y_train_2d = np.array(X_train_3d), np.array(y_train_2d)
    X_test_3d, y_test_2d = np.array(X_test_3d), np.array(y_test_2d)
    X_forecast_3d = np.array(X_forecast_3d)

    X_3d = np.reshape(X_3d, (X_3d.shape[0], X_3d.shape[1], X_3d.shape[2]))
    X_train_3d = np.reshape(X_train_3d, (X_train_3d.shape[0], X_train_3d.shape[1], X_train_3d.shape[2]))
    X_test_3d = np.reshape(X_test_3d, (X_test_3d.shape[0], X_test_3d.shape[1], X_test_3d.shape[2]))
    X_forecast_3d = np.reshape(X_forecast_3d, (X_forecast_3d.shape[0], X_forecast_3d.shape[1], X_forecast_3d.shape[2]))

    return X_3d, y_2d, X_train_3d, y_train_2d, X_test_3d, y_test_2d, X_forecast_3d

In [ ]:
X_3d, y_2d, X_train_3d, y_train_2d, X_test_3d, y_test_2d, X_forecast_3d = packager(X_scaled, X_train_scaled, X_test_scaled, X_forecast_scaled,  24)

In [ ]:
print(f"X_3d Shape : {X_3d.shape}")
print(f"y_2d Shape : {y_2d.shape}\n\n")
print(f"X_train_3d Shape : {X_train_3d.shape}")
print(f"y_train_2d Shape : {y_train_2d.shape}\n\n")
print(f"X_test_3d Shape : {X_test_3d.shape}")
print(f"y_test_2d Shape : {y_test_2d.shape}\n\n")
print(f"X_forecast_3d Shape : {X_forecast_3d.shape}")
# print(f"y_future Shape : {y_future.shape}")

In [ ]:
# # Target label + Multi features and NO---ovlap

# time_steps = 24

# def packager(X_scaled, X_forecast_scaled ,X_train_scaled, X_test_scaled, time_steps=24):
#     X_3d, y_2d, X_forecast_3d , y_forecast ,X_train_3d, y_train_2d, X_test_3d, y_test_2d = [], [], [], [], [], [] , [], []

#     for i in range(0, len(X_scaled)-time_steps, time_steps):
#         X_3d.append(X_scaled[i:i+time_steps, :])
#         X_3d.append(X_scaled[i+time_steps, 0])
        
#     for i in range(0, len(X_forecast_scaled)-time_steps, time_steps):
#         X_forecast_3d.append(X_forecast_scaled[i:i+time_steps, :])
#         y_forecast.append(X_forecast_scaled[i+time_steps, 0])

#     for i in range(0, len(X_train_scaled)-time_steps, time_steps):
#         X_train_3d.append(X_train_scaled[i:i+time_steps, :])
#         y_train_2d.append(X_train_scaled[i+time_steps, 0])

#     for i in range(0, len(X_test_scaled)-time_steps, time_steps):
#         X_test_3d.append(X_test_scaled[i:i+time_steps, :])
#         y_test_2d.append(X_test_scaled[i+time_steps, 0])

#     X_3d, y_2d = np.array(X_3d), np.array(y_2d)
#     X_forecast_3d, y_forecast = np.array(X_forecast_3d), np.array(y_forecast)
#     X_train_3d, y_train_2d = np.array(X_train_3d), np.array(y_train_2d)
#     X_test_3d, y_test_2d = np.array(X_test_3d), np.array(y_test_2d)

#     return X_3d, y_2d, X_forecast_3d, y_forecast, X_train_3d, y_train_2d, X_test_3d, y_test_2d

# Model Time

# Final Model

In [ ]:
import tensorflow as tf

# Seed ayarlamak
seed_value = 53
tf.keras.utils.set_random_seed(seed_value)

# Manual Modelling

In [ ]:
print(f"""
      Model eğitiminde kullandığımız dataların shapeleri
      X_3d Shape : {X_3d.shape}
      y_2d Shape : {y_2d.shape}
      X_test_3d Shape : {X_test_3d.shape}
      y_test_2d Shape : {y_test_2d.shape}
      """)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense , BatchNormalization , Dropout , GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau , EarlyStopping
from keras.optimizers import Adam , Adagrad , Adadelta , Nadam , RMSprop
from keras.losses import MeanAbsolutePercentageError


# Model oluşturma
model = Sequential()
model.add(LSTM(256, activation='tanh' , input_shape = (X_3d.shape[1] , X_3d.shape[2]) ,return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(LSTM(128, activation='tanh' ,return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(64, activation='tanh' ,return_sequences=False))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(32, activation='relu'))
# model.add(GlobalAveragePooling1D())
# model.add(LSTM(12, activation='tanh' ,return_sequences=False))
# model.add(LSTM(4, activation='tanh'  , return_sequences=False))
model.add(Dense(24))

opt = Adam(learning_rate = 0.01)
model.compile(loss= MeanAbsolutePercentageError(), optimizer=opt)


# reduce_lr = ReduceLROnPlateau(factor = 0.1 , patience=8)
# early_stop = EarlyStopping(monitor='val_loss' , patience=7)
model_check = ModelCheckpoint(filepath=r'C:\Users\Emincan\Desktop\Gdz\gdz-competition\Checkpoints\justFeatures_wOverlap\\24-preds\\64d0.2-32d0.2-24dense(lr001-batch32)\\model-{epoch:03d}-{val_loss:.5f}.h5', monitor="val_loss", verbose=1)

# # Modeli eğitme
history = model.fit(X_3d, y_2d, epochs=20, batch_size= 32, validation_split=0.1, callbacks= [model_check] , shuffle=False)


# Modeli değerlendirme
mape_score = model.evaluate(X_test_3d, y_test_2d)
print('Test MAPE: %.3f' % mape_score)

### Plot the Model History

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

plt.show()

## Import the Model

In [ ]:
from tensorflow.keras.models import load_model

model = load_model(r"C:\Users\Emincan\Desktop\Gdz\gdz-competition\Checkpoints\justFeatures_NOovlap\256-128-64-32d-24d(lr0001-batch32)\\model-038-12.65707.h5")

# Eval Metrics

In [ ]:
from eval_metrics import eval_metrics

y_pred = model.predict(X_test_3d)

eval_metrics(y_test_2d , y_pred)

In [ ]:
X_test_3d.shape

In [ ]:
y_test_2d.shape

# Prediction